In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
doc2vec_path = "/content/drive/MyDrive/Research/Political Image Debiasing/doc2vec/"
dataset_path = "/content/drive/MyDrive/Research/Political Image Debiasing/dataset/"
doc2vec_model_path = os.path.join(doc2vec_path, "doc2vec_model.gensim")
doc_feats_knn_path = os.path.join(doc2vec_path, "document_feats_knn.pickle")
split_paths = os.path.join(dataset_path, "train_test_paths_small.pickle")
bias_dataset_path = "/content/drive/MyDrive/Research/Political Image Debiasing/dataset/newdataset_withBias.csv"

In [3]:
BIAS_TOLERANCE = 0.1

In [4]:
# Open CSV file with biases, get all imagepath/bias pairs into a dictionary
import pandas as pd
df = pd.read_csv(bias_dataset_path)
img_bias_dict = {}
for index, row in df.iterrows():
    img_bias_dict[row["img_filepath"]] = row["bias_score"]

def img_to_bias(img_path):
    return img_bias_dict[img_path]

print(df)
print(img_to_bias("new_imgs/0.jpeg"))
print(img_to_bias("new_imgs/1.jpeg"))
print(img_to_bias("new_imgs/199999.jpeg"))


        Unnamed: 0          img_filepath              website  \
0                0       new_imgs/0.jpeg         observer.com   
1                1       new_imgs/1.jpeg        world.wng.org   
2                2       new_imgs/2.jpeg      unionleader.com   
3                3       new_imgs/3.jpeg         trtworld.com   
4                4       new_imgs/4.jpeg        world.wng.org   
...            ...                   ...                  ...   
199995      199995  new_imgs/199995.jpeg       elitedaily.com   
199996      199996  new_imgs/199996.jpeg         juancole.com   
199997      199997  new_imgs/199997.jpeg            revcom.us   
199998      199998  new_imgs/199998.jpeg  countercurrents.org   
199999      199999  new_imgs/199999.jpeg     palmerreport.com   

                                                     text  bias_score  
0       During last night's debut of the HBO documenta...        0.40  
1       Eugene Vodolazkin's Laurus was a finalist for ...        0.60  
2  

In [5]:
# !unzip "/content/drive/MyDrive/Research/Political Image Debiasing/dataset/new_imgs.zip" -d "/content/drive/MyDrive/Research/Political Image Debiasing/dataset/"

In [6]:
!unzip "/content/drive/MyDrive/Research/Political Image Debiasing/dataset/new_imgs.zip" -d /content

Streaming output truncated to the last 5000 lines.
  inflating: /content/new_imgs/149514.jpeg  
  inflating: /content/__MACOSX/new_imgs/._149514.jpeg  
  inflating: /content/new_imgs/38002.jpeg  
  inflating: /content/__MACOSX/new_imgs/._38002.jpeg  
  inflating: /content/new_imgs/163956.jpeg  
  inflating: /content/__MACOSX/new_imgs/._163956.jpeg  
  inflating: /content/new_imgs/104492.jpeg  
  inflating: /content/__MACOSX/new_imgs/._104492.jpeg  
  inflating: /content/new_imgs/45532.jpeg  
  inflating: /content/__MACOSX/new_imgs/._45532.jpeg  
  inflating: /content/new_imgs/13352.jpeg  
  inflating: /content/__MACOSX/new_imgs/._13352.jpeg  
  inflating: /content/new_imgs/103357.jpeg  
  inflating: /content/__MACOSX/new_imgs/._103357.jpeg  
  inflating: /content/new_imgs/14497.jpeg  
  inflating: /content/__MACOSX/new_imgs/._14497.jpeg  
  inflating: /content/new_imgs/155537.jpeg  
  inflating: /content/__MACOSX/new_imgs/._155537.jpeg  
  inflating: /content/new_imgs/73953.jpeg  
  in

In [7]:
import os
import pickle
FOLDER_PATH = '/content/new_imgs/'
print(len(os.listdir(os.path.join(FOLDER_PATH))))
count = 0
for asdf in os.listdir(os.path.join(FOLDER_PATH)):
  count += 1
  print(asdf)
  if count%10 == 0:
    break

for asdf in os.listdir(os.path.join(FOLDER_PATH)):
  if (asdf == "6653_121.jpeg"):
    print(f"{asdf} is here")
    break

dataset = pickle.load(open("/content/drive/MyDrive/Research/Political Image Debiasing/dataset/dataset_metadata_clean_5000.pickle", "rb"))
for leaning in dataset.keys():
  for topic in dataset[leaning].keys():
    for sample in dataset[leaning][topic]:
      print(sample)
      break
    break
  break

local_path_to_trupath = {}
trupath_to_local_path = {}
count = 0
basepath = "/content/new_imgs/"
for leaning in dataset.keys():
  for topic in dataset[leaning].keys():
    for sample in dataset[leaning][topic]:
      local_path_to_trupath[sample["local_path"]] = os.path.join(basepath, str(count)+".jpeg")
      trupath_to_local_path[os.path.join(basepath, str(count)+".jpeg")] = sample["local_path"]
      count += 1
print(len(local_path_to_trupath))
print(len(trupath_to_local_path))

def impath_from_local(local_path):
  return local_path_to_trupath[str(local_path)]

def local_from_impath(impath):
  return trupath_to_local_path[str(impath)]

200001
38816.jpeg
104509.jpeg
143514.jpeg
28947.jpeg
147950.jpeg
33222.jpeg
85309.jpeg
1453.jpeg
195219.jpeg
10525.jpeg
{'local_path': 'imgs/right/Homelessness/observer.com/7902_163.jpg', 'image_description': "Diana, Our Mother' Offers Insight Into What Growing Up Royal Means ...", 'origin_webpage': 'http://observer.com/2017/07/princess-diana-hbo-documentary-pictures-and-quotes/', 'web_source': 'observer.com', 'content_text': "During last night's debut of the HBO documentary,  Diana, Our Mother: Her Life and Legacy,  the sons of the late princess opened up about what growing up with her was like. By offering anecdotes from their mother's life, including pranks and philanthropy projects, dad dancing  Prince William and the formerly rebellious  Prince Harry offered a closer look at their royal lives, all while honoring of the 20th anniversary of Princess Diana's death."}
199996
200000


In [8]:
# Load KNN - THIS IS PUT INTO SHARED MEMORY FOR MULTIPROCESSING
import pickle
import numpy as np
neighbors = pickle.load(open(doc_feats_knn_path, 'rb'))
_neighbors_pths2idx = {v : k for k,v in enumerate(neighbors['paths'])} # SHARED!
_neighbors_idxs, _neighbors_dists = zip(*neighbors['neighbors'])
_neighbors_idxs = np.stack([np.pad(a.ravel(), (0, 200 - a.size), 'constant', constant_values=0) for a in _neighbors_idxs], axis=0) # SHARED
_neighbors_dists = np.stack([np.pad(a.ravel(), (0, 200 - a.size), 'constant', constant_values=0) for a in _neighbors_dists], axis=0)  # SHARED
_neighbors_pths = neighbors['paths'] # SHARED
print(len(_neighbors_idxs))
print(_neighbors_idxs[0])
# print(_neighbors_dists)
# print(max(_neighbors_dists[0]))
# print(max(_neighbors_dists[1]))
# print(max(_neighbors_dists[2]))
# print(max(_neighbors_dists[3]))


_neighbors_pths2idx_bias = {v : k for k,v in enumerate(neighbors['paths'])} # SHARED!
_neighbors_idx2pth_bias = {k : v for k,v in enumerate(neighbors['paths'])} # SHARED!
print(_neighbors_pths2idx_bias)

def bias_from_idx(idx):
  impath = impath_from_local(_neighbors_idx2pth_bias[idx]).replace("/content/", "")
  return img_to_bias(impath)

unique_bias_scores = np.unique([b for b in img_bias_dict.values()])
# For each unique bias score, get all the indices that match that bias
bias_to_indices = {b: [i for i in _neighbors_pths2idx.values() if bias_from_idx(i)==b] for b in unique_bias_scores}
print(bias_to_indices)
def indexes_from_bias(bias):
  indices = []
  # Get all bias scores that are within range of input bias
  relevant_scores = unique_bias_scores[np.where(abs(unique_bias_scores-bias)<=BIAS_TOLERANCE)]
  # Extend list of indices with all of them
  for s in relevant_scores:
    indices.extend(bias_to_indices[s])
  return list(set(indices))

# print(neighbors)
# _neighbors_idxs_bias, _neighbors_dists_bias = zip(*neighbors['neighbors'])
# # print(_neighbors_idxs_bias[0])
# # print(_neighbors_dists_bias[0])
# _neighbors_idxs_bias = np.stack([np.pad(a.ravel(), (0, 200 - a.size), 'constant', constant_values=0) for a in _neighbors_idxs_bias], axis=0) # SHARED
# _neighbors_dists_bias = np.stack([np.pad(a.ravel(), (0, 200 - a.size), 'constant', constant_values=0) for a in _neighbors_dists_bias], axis=0)  # SHARED
# print(np.array(_neighbors_idxs_bias[0]))
# print(np.array(_neighbors_dists_bias[0]))
# _neighbors_biases = np.stack([np.pad([b for b in a if abs(bias_from_idx(b)-bias_from_idx(idx))<=BIAS_TOLERANCE], (0, 200 - a.size), 'constant', constant_values=0) for idx, a in enumerate(_neighbors_idxs_bias)], axis=0)  # SHARED
# _neighbors_biases = np.array([[b for b in a if abs(bias_from_idx(b)-bias_from_idx(idx))<=BIAS_TOLERANCE] for idx, a in enumerate(_neighbors_idxs_bias)])  # SHARED
# "Bias neighbors" are ALL images/indexes that are within BIAS_TOLERANCE of the image
# _neighbors_biases = np.array([[b for b in _neighbors_pths2idx_bias.values() if idx!=b and abs(bias_from_idx(b)-bias_from_idx(idx))<=BIAS_TOLERANCE] for idx, a in enumerate(_neighbors_idxs_bias)])  # SHARED
all_bias_neighbors = []
for idx in _neighbors_pths2idx.values():
  bias_neighbors = indexes_from_bias(bias_from_idx(idx))
  bias_neighbors.remove(idx)
  all_bias_neighbors.append(bias_neighbors)
_neighbors_biases = all_bias_neighbors  # SHARED

print(_neighbors_biases[0])
print(f"len neighbor biases {len(_neighbors_biases)}")
print(f"min neighborhood within bias {min([len(x) for x in _neighbors_biases])}")
_neighbors_pths_bias = neighbors['paths'] # SHARED
print([x for x in neighbors.keys()])
print(neighbors["neighbors"][0])
print(_neighbors_pths_bias)

del neighbors  # Free python object - use only Numpy variants

23356
[13362  1924 19771 12847  4456 14983  8212  6200 12326 14783  5162 15536
 19892  4074  7986  6041  2970  2429 22832 20777 16000 11910  9423 21924
 20334  8843 18122   992 14939 17378 13288  4466  1638  3209  8819  5637
 14277  8476  1964 19006 14660  7578 14888  7347 19776 12738 15991 12137
 17307 17674 21966  8765  6658  8050  9496 18821  6880 19456 14970 20733
 18555  4502 13282 13835 11031  6293 19626 12587 13640 11075  7736 19087
 11420 18529 17336 21986   515 22593  9355 11834  3237  8403 19092 22805
 15466  6682 12537 16867 17827  3477 17640 16743 14527 10270 19002   170
 10239  4136  1277  6258  6187 11532  8702 21155 11292  1968 18940  5067
 21304 14139 21283  7575 22016 18124 22762  7783 21322 22655  7866 13236
  3077 18724   327  8489  3736 12700 12174  4218 21911  7810 15151 22314
  5662 18548 14721 18988 10781 14661 11191 20209 11926  4334    84 14743
 17728 18651   473  7609 14354  6617   904 13734 10472 20236 17632 19629
 14917  4757  5412  4471 22595  3557  8754  3

## Remake Train/Test Splits (Only Once)

In [9]:
# # Remaking the train/test splits with the new (smaller) dataset
# import random
# import pickle
# def remake_train_test_split(test_samples_per_topic=500):
#   dataset = pickle.load(open("/content/drive/MyDrive/Research/Political Image Debiasing/dataset/dataset_metadata_clean_5000.pickle", "rb"))
#   splits_dict = {
#       "train": [],
#       "test": [],
#   }
#   for leaning in dataset.keys():
#     for topic in dataset[leaning].keys():
#       test_samples = [s["local_path"] for s in random.sample(dataset[leaning][topic], test_samples_per_topic)]
#       train_samples = [s["local_path"] for s in dataset[leaning][topic] if s["local_path"] not in test_samples]
#       splits_dict["test"].extend(test_samples)
#       splits_dict["train"].extend(train_samples)
#   return splits_dict

# new_splits = remake_train_test_split()
# print(len(new_splits["train"]))
# print(len(new_splits["test"]))
# f = open(os.path.join(dataset_path, "train_test_paths_small.pickle"), "wb")
# pickle.dump(new_splits, f)
# f.close()

## Training Utils

In [10]:
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 3.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-vgf430_7
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-vgf430_7
  Resolved https://github.com/openai/CLIP.git to commit a9b1bf5920416aaeaec965c25dd9e8f98c864f16
  Preparing metadata (setup.py) ... done
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369398 sha256=791cff917f67e7ce9fe3dba7e0e201351b6ef70f8c78444f7c263bc0a1502bce
  Stored in directory: /tmp/pip-ephem-wheel-cache-fdozu5du/wheels/da/2b/4c/d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4
Successfully built clip


In [11]:

import torch
import clip
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"

last_checkpoint = "/content/drive/MyDrive/Research/Political Image Debiasing/models/l_i2t_l_sym_1_l_img_0_3_l_text_0_2_l_bias_0_3/clip_model3.pt"

def get_clip_components(model_name="ViT-B/32", download_root="./.cache/clip"):
    model, preprocess = clip.load(model_name, device=device, download_root=download_root)
    return (model.visual, model.transformer, preprocess)

def get_clip(model_name="ViT-B/32", download_root="./.cache/clip"):
    model, preprocess = clip.load(model_name, device=device, download_root=download_root, jit=False)
    # Check for loading model
    if os.path.isfile(last_checkpoint):
        if (torch.cuda.is_available()):
          loaded = torch.load(last_checkpoint)
        else:
          loaded = torch.load(last_checkpoint, map_location=torch.device('cpu'))
        model_state = loaded["model_state_dict"]
        model.load_state_dict(model_state)
        print(f"Loaded model state dict: {last_checkpoint}")
    model = model.float()
    return (model, preprocess)

def load_optimizer_dict(optimizer):
    if os.path.isfile(last_checkpoint):
        loaded = torch.load(last_checkpoint)
        opt_state = loaded["optimizer_state_dict"]
        optimizer.load_state_dict(opt_state)
        print(f"Loaded optimizer state dict: {last_checkpoint}")
    return optimizer

def get_last_epoch():
    if os.path.isfile(last_checkpoint):
        loaded = torch.load(last_checkpoint)
        return loaded["epoch"]
    return -1

In [12]:
import csv
import pickle
import random
import nltk
import sys
from gensim.parsing.preprocessing import preprocess_string, strip_tags, strip_punctuation, strip_multiple_whitespaces, remove_stopwords, strip_short, strip_non_alphanum
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import numpy as np

# csv.field_size_limit(sys.maxsize)
# Above line breaks on my machine, changing to try except based on
# https://stackoverflow.com/questions/15063936/csv-error-field-larger-than-field-limit-131072
try:
    csv.field_size_limit(sys.maxsize)
except OverflowError:
    maxInt = int(sys.maxsize/10)
d2v = Doc2Vec.load(doc2vec_model_path)
word_to_vector = d2v.wv
# word_to_index = {word : idx for idx, word in enumerate(word_to_vector.index2entity)}
word_to_index = {word : idx for idx, word in enumerate(word_to_vector.index_to_key)}
del d2v
word_to_index['<!START!>'] = len(word_to_index)
word_to_index['<!END!>'] = len(word_to_index)

def get_db(datapath):
    # db = pickle.load(open('complete_db.pickle', 'rb'))
    f = open(datapath, 'rb')
    db = pickle.load(f)
    f.close()
    orig_paths_and_text = []
    for politics, issues in db.items():
        for issue, items in issues.items():
            for item in items:
                # if len(item['content_text']) > 500:
                # I changed this because I already cut down articles to 2 sentences
                if len(item['content_text']) > 10:
                    orig_paths_and_text.append(
                        (item['local_path'], item['content_text']))
    random.shuffle(orig_paths_and_text)
    return orig_paths_and_text
    
def convert_text(tup, sentence_limit=2):
    def normalize_text(text):
        text = text.encode('ascii', errors='replace').decode('ascii')
        return text
    pth, text = tup
    text = normalize_text(text)
    # Sentence limit
    sentences = []
    for sentence in nltk.tokenize.sent_tokenize(text)[:sentence_limit]:
        sentences.append(sentence)
    text = ' '.join(sentences)
    CUSTOM_FILTERS = [lambda x: x.lower(), strip_tags, strip_punctuation, strip_multiple_whitespaces, strip_non_alphanum, remove_stopwords, strip_short]
    tokenized_text = preprocess_string(text, CUSTOM_FILTERS)
    tokenized_text = list(filter(lambda x: x in word_to_index, tokenized_text))
    return (pth, tokenized_text)

## Dataset

In [13]:
import pickle
import random
import torch
import time
import numpy as np

from PIL import Image
from multiprocessing import Pool
from tqdm import tqdm
from torchvision import transforms

import clip

# from train_utils import get_db, convert_text
# from train_utils import _neighbors_pths2idx, word_to_index, _neighbors_idxs, _neighbors_pths, _neighbors_dists

import os
from pathlib import Path

class MyDataset(torch.utils.data.Dataset):
    result_db = {}
    train_set = []
    val_set = []
    def __init__(self, mode, datapath):
        super().__init__()
        # self.base_path = dataset_path
        self.base_path = "/content/new_imgs/"
        if not MyDataset.result_db:
            # train_test_dict = pickle.load(open('train_test_pths.pickle', 'rb'))
            train_test_dict = pickle.load(open(split_paths, 'rb'))
            train_set = train_test_dict['train']
            val_set = set(random.sample(train_set, len(train_test_dict['test'])))
            train_set = [pth for pth in train_set if pth not in val_set]
            MyDataset.train_set = list(train_set)
            MyDataset.test_set = list(val_set)
            complete_db = [(pth, txt[:10000]) for pth, txt in get_db(datapath) if pth in _neighbors_pths2idx and (pth in MyDataset.train_set or pth in MyDataset.test_set)]
            pool = Pool(processes=4)
            documents_to_words = dict(tqdm(pool.imap_unordered(convert_text, complete_db, chunksize=1), total=len(complete_db), leave=False, desc='Convert to Fixed Dict'))
            pool.close()
            # Convert words to GT word vector
            for pth, sentence in tqdm(documents_to_words.items(), leave=False, desc='Words to Vals'):
                if not sentence:
                    continue
                # Tokenization of sentence
                # print(sentence)
                # sentence_to_idx = np.asarray([word_to_index['<!START!>']]+[word_to_index[word] for word in sentence]+[word_to_index['<!END!>']], dtype=np.int32)
                # sentence_to_idx = np.asarray([word_to_index[word] for word in sentence])
                # sentence_to_idx = " ".join(sentence)
                sentence_to_idx = " ".join(sentence[:75]) # max clip text length is 77, must leave room for start and end tokens as well
                while True:
                    try:
                      clip.tokenize([sentence_to_idx])
                      break
                    except:
                      sentence_to_idx = sentence_to_idx[:-1]
                      pass
                # print(sentence_to_idx)
                MyDataset.result_db[pth] = sentence_to_idx
            ##########################################################
        if mode == 'train':
            self.dataset = [img for img in MyDataset.train_set if img in MyDataset.result_db]
            self.transform = transforms.Compose([
                transforms.Resize(size=(224, 224)),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])])
        elif mode == 'val':
            self.dataset = [img for img in MyDataset.test_set if img in MyDataset.result_db]
            self.transform = transforms.Compose([
                transforms.Resize(size=(224, 224)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])])
    def __len__(self):
        return len(self.dataset)
    def __getitem__(self, index):
        while True:
            try:
                # imgpath = os.path.join(self.base_path, self.dataset[index])
                # imgpath = os.path.join(self.base_path, Path(self.dataset[index]).name)
                imgpath = impath_from_local(self.dataset[index])
                # print(f"trying to get img {imgpath}")
                img = Image.open(imgpath).convert('RGB')
                # print(f"got img at {imgpath}")
                s = time.time()
                neighbor_data = get_doc2vec_neighbors(self.dataset[index], self.transform, img, self.base_path)
                ne = time.time()-s
                # print(f"got neighbor data for {imgpath}")
                s = time.time()
                bias_neighbor_data = get_doc2vec_neighbors_bias(self.dataset[index], self.transform, img, self.base_path)
                bne = time.time()-s
                # print(f"neighbor data time: {ne},  bias neighbor data time: {bne}")
                # print(f"got bias neighbor data for {imgpath}")
                break
            except Exception as e:
                print(e)
                index = random.randrange(len(self.dataset))
                continue        
        # img, path, neighbor_img, neighbor_pth        
        return self.transform(img), imgpath, neighbor_data[0], neighbor_data[1], bias_neighbor_data[0], bias_neighbor_data[1]
    

def get_doc2vec_neighbors(pth, transform, orig_img, base_path):
    assert(pth in _neighbors_pths2idx)    
    root_idx = _neighbors_pths2idx[pth]
    root_neighbor_idxs = _neighbors_idxs[root_idx, :]
    root_neighbor_dists = _neighbors_dists[root_idx, :]
    non_dupe_idxs = np.nonzero(np.abs(root_neighbor_dists) > 0)[0].tolist()
    neighbor_imgs = []
    neighbor_pths = []
    non_dupe_idxs = [ndi for ndi in non_dupe_idxs if _neighbors_pths[root_neighbor_idxs[ndi]] in MyDataset.train_set and _neighbors_pths[root_neighbor_idxs[ndi]] in MyDataset.result_db]
    # Choose one randomly from first N (here we use N=10 most similar neighbors - worked slightly in later tests) but you can also adjust to all 200
    non_dupe_idxs = non_dupe_idxs[:10]
    random.shuffle(non_dupe_idxs)
    # IF YOU WANT - YOU CAN USE MORE THAN ONE NEIGHBOR
    for ndi in non_dupe_idxs:
        neighbor_idx = root_neighbor_idxs[ndi]
        neighbor_pth = _neighbors_pths[neighbor_idx]
        # print(f"ndi {ndi}")
        # print(f"neighborpath {neighbor_pth}")

        # Try commenting out
        if neighbor_pth not in MyDataset.train_set or root_idx == neighbor_idx or neighbor_pth not in MyDataset.result_db:
            continue

        try:            
            # imgpath = os.path.join(base_path, "new_imgs", Path(neighbor_pth).name)
            # imgpath = os.path.join(base_path, Path(neighbor_pth).name)
            # print(f"p {Path(neighbor_pth)}")
            # print(f"pname {Path(neighbor_pth).name}")
            imgpath = impath_from_local(Path(neighbor_pth))
            # print(f"nei img pat {imgpath}")
            img = transform(Image.open(imgpath).convert('RGB'))
            # print("HERHE")
            # img = transform(Image.open(os.path.join(base_path, neighbor_pth)).convert('RGB'))
        except Exception as e:
            # print(e)
            continue
        neighbor_imgs.append(img)
        neighbor_pths.append(neighbor_pth)
    # USING 1 NEIGHBOR
    # print(neighbor_imgs)
    return (neighbor_imgs[0], neighbor_pths[0])

def get_doc2vec_neighbors_bias(pth, transform, orig_img, base_path):
    assert(pth in _neighbors_pths2idx_bias)    
    root_idx = _neighbors_pths2idx_bias[pth]
    root_neighbor_idxs = _neighbors_biases[root_idx] # Get all bias neighbors of root
    # root_neighbor_dists = _neighbors_dists_bias[root_idx, :]
    # non_dupe_idxs = np.nonzero(np.abs(root_neighbor_dists) > 0)[0].tolist()
    neighbor_imgs = []
    neighbor_pths = []
    # print("HERE")
    
    s = time.time()
    non_dupe_idxs = random.sample(root_neighbor_idxs, 30)
    asdf = time.time()-s
    s = time.time()
    non_dupe_idxs = [ndi for ndi in non_dupe_idxs if _neighbors_idx2pth_bias[ndi] in MyDataset.train_set and _neighbors_idx2pth_bias[ndi] in MyDataset.result_db]
    ndilc = time.time()-s
    # print("THERE")
    # Choose randomly from 10 random bias neighbors
    s = time.time()
    non_dupe_idxs = random.sample(non_dupe_idxs, 10)
    sa = time.time()-s
    s=time.time()
    random.shuffle(non_dupe_idxs)
    sh = time.time()-s
    # print("post shuffle")
    # IF YOU WANT - YOU CAN USE MORE THAN ONE NEIGHBOR
    s=time.time()
    for ndi in non_dupe_idxs:
        neighbor_idx = ndi
        neighbor_pth = _neighbors_idx2pth_bias[neighbor_idx]
        # print(f"ndi {ndi}")
        # print(f"neighborpath {neighbor_pth}")

        # Try commenting out
        if neighbor_pth not in MyDataset.train_set:
            print(f"{neighbor_pth} not in train set")
        if root_idx == neighbor_idx:
            print(f"{root_idx} == {neighbor_idx}. skipping.")
        if neighbor_pth not in MyDataset.result_db:
            print(f"{neighbor_pth} not in result_db")
        if neighbor_pth not in MyDataset.train_set or root_idx == neighbor_idx or neighbor_pth not in MyDataset.result_db:
            continue

        try:            
            # imgpath = os.path.join(base_path, "new_imgs", Path(neighbor_pth).name)
            # imgpath = os.path.join(base_path, Path(neighbor_pth).name)
            # print(f"p {Path(neighbor_pth)}")
            # print(f"pname {Path(neighbor_pth).name}")
            imgpath = impath_from_local(Path(neighbor_pth))
            # print(f"nei img pat {imgpath}")
            img = transform(Image.open(imgpath).convert('RGB'))
            # print("HERHE")
            # img = transform(Image.open(os.path.join(base_path, neighbor_pth)).convert('RGB'))
        except Exception as e:
            # print(e)
            continue
        neighbor_imgs.append(img)
        neighbor_pths.append(neighbor_pth)
    lt = time.time()-s

    # print(f"initsample: {asdf},  nondupelistcomp: {ndilc},  sampletime: {sa},  shuffletime: {sh},  looptime: {lt}")
    # USING 1 NEIGHBOR
    # print(neighbor_imgs)
    return (neighbor_imgs[0], neighbor_pths[0])

## Loss

In [14]:
import torch

def angular_loss(anchors, positives, negatives, angle_bound=1.):
    """
    Calculates angular loss
    :param anchors: A torch.Tensor, (n, embedding_size)
    :param positives: A torch.Tensor, (n, embedding_size)
    :param negatives: A torch.Tensor, (n, embedding_size)
    :param angle_bound: tan^2 angle
    :return: A scalar
    """
    anchors = torch.unsqueeze(anchors, dim=1)  # (n, 1, embedding_size)
    positives = torch.unsqueeze(positives, dim=1)  # (n, 1, embedding_size)
    negatives = torch.unsqueeze(negatives, dim=0).expand(len(positives), -1, -1)

    # print(anchors.dtype)
    # print(positives.dtype)
    # print(negatives.dtype)
    # print(f"anchors is nans {anchors.isnan().any()}")
    # print(f"positives is nans {positives.isnan().any()}")
    # print(f"negatives is nans {negatives.isnan().any()}")

    x = 4. * angle_bound * torch.matmul((anchors + positives), negatives.transpose(1, 2)) \
        - 2. * (1. + angle_bound) * torch.matmul(anchors, positives.transpose(1, 2))  # (n, 1, n-1)

    # Preventing overflow
    with torch.no_grad():
        t = torch.max(x, dim=2)[0]

    x = torch.exp(x - t.unsqueeze(dim=1))
    x = torch.log(torch.exp(-t) + torch.sum(x, 2))
    # print(f"loss is nans {x.isnan().any()}")
    loss = torch.mean(t + x)
    # print(f"indiv loss: {loss}")

    return loss

def bias_loss(anchors, positives, negatives):
    raise NotImplementedError()

## Training Code

In [15]:
# FROM: https://github.com/CLT29/semantic_neighborhoods
# The first parameter is the weight of the symmetric retrieval constraint (text to image retrieval).
# The second weight is the within-modality image retrieval constraint (image -> image neighbors, i.e. L_img), 
# and the third is the text within-modality constraint, i.e. L_text.
# https://github.com/CLT29/semantic_neighborhoods
T2I_WEIGHT = 1
I_WEIGHT = 0.3
T_WEIGHT = 0.2
B_WEIGHT = 0.3
# last is the media bias weight

NUM_WORKERS = 2

In [16]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import nltk
import gensim
import numpy as np
import pickle
import re, math
import os
import sys
import json
import random
import glob
import gzip
import time
import traceback
import torch
import json
from torch.autograd import Variable
from torchvision import transforms, models
from enum import Enum
from PIL import Image
import torch.nn.functional as F
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
import sklearn.utils
from collections import Counter
import csv, itertools
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import normalize
from multiprocessing import Pool

nltk.download('punkt')

# from train_utils import get_db, convert_text
# from train_utils import word_to_index, word_to_vector, _neighbors_pths2idx

# from train_loss import angular_loss, bias_loss

# from clip_utils import get_clip_components, get_clip

# from dataset import MyDataset

# TODO:
# -argparse instead of sys.argv
#     - 3 original inputs
#     -lr
#     -batch_size
#     -num_workers
# - replace img_model and rnn_model with clip encoders
# - finish new loss function
# CHECK:
# Saving/loading https://github.com/openai/CLIP/issues/83

do_train = False

def main():
    if not do_train:
      return

    clipmodel, clippreprocess = get_clip()
    vision_encoder, text_encoder, preprocess = get_clip_components()
    print("Getting train loader.")
    # train_dataloader = torch.utils.data.DataLoader(dataset=MyDataset('train', "E://Research//Political_Image_Debiasing//dataset//dataset_metadata_clean_5000.pickle"), batch_size=4, shuffle=True, num_workers=4)
    train_dataloader = torch.utils.data.DataLoader(dataset=MyDataset('train', "/content/drive/MyDrive/Research/Political Image Debiasing/dataset/dataset_metadata_clean_5000.pickle"), batch_size=4, shuffle=True, num_workers=NUM_WORKERS)
    print("Getting val loader.")
    test_dataloader = torch.utils.data.DataLoader(dataset=MyDataset('val', "/content/drive/MyDrive/Research/Political Image Debiasing/dataset/dataset_metadata_clean_5000.pickle"), batch_size=4, shuffle=False, num_workers=NUM_WORKERS)
    print("Done loading datasets")
    writer = SummaryWriter(f'/content/drive/MyDrive/Research/Political Image Debiasing/models/l_i2t_l_sym_{str(T2I_WEIGHT).replace(".","_")}_l_img_{str(I_WEIGHT).replace(".","_")}_l_text_{str(T_WEIGHT).replace(".","_")}_l_bias_{str(B_WEIGHT).replace(".","_")}/')
    # img_model = torch.nn.DataParallel(ImageModel()).cuda()
    # rnn_model = torch.nn.DataParallel(RecurrentModel()).cuda()
    img_model = torch.nn.DataParallel(vision_encoder)
    rnn_model = torch.nn.DataParallel(text_encoder)
    if (torch.cuda.is_available()):
      img_model = img_model.cuda()
      rnn_model = rnn_model.cuda()
    img_model.train()
    rnn_model.train()

    # clipmodel = torch.nn.DataParallel(clipmodel)
    if (torch.cuda.is_available()):
      clipmodel = clipmodel.cuda()
    clipmodel.train()

    last_epoch = get_last_epoch()

    # optimizer = torch.optim.Adam(params=itertools.chain(img_model.parameters(), rnn_model.parameters()), lr=0.0001, weight_decay=1e-5)
    optimizer = torch.optim.Adam(params=itertools.chain(clipmodel.parameters()), lr=0.0001, weight_decay=1e-5)
    optimizer = load_optimizer_dict(optimizer)
    ### LR SCHEDULER
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', verbose=True, patience=5)
    ###
    itr = 0    
    best_loss = sys.maxsize
    # torch.autograd.set_detect_anomaly(True)
    for e in tqdm(range(1, 1000), ascii=True, desc='Epoch'):
        if e <= last_epoch:
            continue
        img_model.train()
        rnn_model.train()
        clipmodel.train()
        random.seed()
        with tqdm(total=len(train_dataloader), ascii=True, leave=False, desc='iter') as pbar:
            for i, (images, paths, neighbor_imgs, neighbor_pths, bias_neighbor_imgs, bias_neighbor_pths) in enumerate(train_dataloader):
                itr += 1
                optimizer.zero_grad()
                if (torch.cuda.is_available()):
                  images = images.float().cuda()
                # print("after images to cuda")
                # image_projections = img_model(images) # Batch size x 256
                image_projections = clipmodel.encode_image(images)
                # print("after images projection")
                if (torch.cuda.is_available()):
                  neighbor_imgs = neighbor_imgs.float().cuda()
                # neighbor_imgs_projections = img_model(neighbor_imgs)
                neighbor_imgs_projections = clipmodel.encode_image(neighbor_imgs)
                # print("after neighbors projection")
                # Get bias neighbor projections
                if (torch.cuda.is_available()):
                  bias_neighbor_imgs = bias_neighbor_imgs.float().cuda()
                bias_neighbor_imgs_projections = clipmodel.encode_image(bias_neighbor_imgs)
                # print("after bias neighbors projection")

                # neighbor_sentences = [torch.from_numpy(MyDataset.result_db[path]).long() for path in neighbor_pths]
                # neighbor_lengths = torch.LongTensor([torch.numel(item) for item in neighbor_sentences])
                # neighbor_sentences = torch.nn.utils.rnn.pad_sequence(sequences=neighbor_sentences, batch_first=True, padding_value=word_to_index['<!END!>'])
                neighbor_sentences = [MyDataset.result_db[path] for path in neighbor_pths]
                # print("after neighbor sentences")
                # positive_sentences = [torch.from_numpy(MyDataset.result_db[local_from_impath(path)]).long() for path in paths]
                # pos_lengths = torch.LongTensor([torch.numel(item) for item in positive_sentences])
                # positive_sentences = torch.nn.utils.rnn.pad_sequence(sequences=positive_sentences, batch_first=True, padding_value=word_to_index['<!END!>'])
                positive_sentences = [MyDataset.result_db[local_from_impath(path)] for path in paths]
                # print("after positive sentences")
                # negative_sentences = [torch.from_numpy(MyDataset.result_db[path]).long() for path in random.sample(train_dataloader.dataset.dataset, len(positive_sentences))]
                # neg_lengths = torch.LongTensor([torch.numel(item) for item in negative_sentences])
                # negative_sentences = torch.nn.utils.rnn.pad_sequence(sequences=negative_sentences, batch_first=True, padding_value=word_to_index['<!END!>'])
                negative_sentences = [MyDataset.result_db[path] for path in random.sample(train_dataloader.dataset.dataset, len(positive_sentences))]
                # if (torch.cuda.is_available()):
                #   neighbor_sentences = neighbor_sentences.cuda()
                #   positive_sentences = positive_sentences.cuda()
                #   negative_sentences = negative_sentences.cuda()
                # print("after negative sentences")
                # print(neighbor_sentences)
                # print(positive_sentences)
                # print(negative_sentences)
                # neighbor_projections = rnn_model(neighbor_sentences, neighbor_lengths)
                neighbor_sentences = clip.tokenize(neighbor_sentences)
                # neighbor_projections = rnn_model(neighbor_sentences)
                # print("after neighbor sentence projections")
                # positive_projections = rnn_model(positive_sentences, pos_lengths)
                positive_sentences = clip.tokenize(positive_sentences)
                # positive_projections = rnn_model(positive_sentences)
                # print("after positive sentence projections")
                # negative_projections = rnn_model(negative_sentences, neg_lengths)
                negative_sentences = clip.tokenize(negative_sentences)
                # negative_projections = rnn_model(negative_sentences)
                # print("after negative sentence projections")

                if (torch.cuda.is_available()):
                  neighbor_sentences = neighbor_sentences.cuda()
                  positive_sentences = positive_sentences.cuda()
                  negative_sentences = negative_sentences.cuda()

                neighbor_projections = clipmodel.encode_text(neighbor_sentences)
                positive_projections = clipmodel.encode_text(positive_sentences)
                negative_projections = clipmodel.encode_text(negative_sentences)

                # Baseline loss
                # Compute n-pairs angular loss
                image_projections_np = torch.repeat_interleave(image_projections, len(negative_projections), dim=0)
                positive_projections_np = torch.repeat_interleave(positive_projections, len(negative_projections), dim=0)
                negative_projections_np = negative_projections.repeat(len(negative_projections), 1)
                l_i2t = angular_loss(anchors=image_projections_np, positives=positive_projections_np, negatives=negative_projections_np)
                # print("after n-pairs angular loss")

                # L_img - Image anchor, Neighbor img anchor, negative image neighbors. Angular Npairs
                image_projections_np = torch.repeat_interleave(image_projections, len(negative_projections), dim=0)
                neighbor_imgs_projections_np = torch.repeat_interleave(neighbor_imgs_projections, len(negative_projections), dim=0)
                permute_idxs = torch.from_numpy(np.asarray([j if i != j else (j+1) % len(image_projections) for i in range(len(image_projections)) for j in range(len(image_projections))]))
                image_projections_np2 = image_projections[permute_idxs,...]
                l_img = angular_loss(anchors=image_projections_np, positives=neighbor_imgs_projections_np, negatives=image_projections_np2)
                # print("after image angular loss")

                # L_text - Angular npairs
                positive_projections_np = torch.repeat_interleave(positive_projections, len(negative_projections), dim=0)
                neighbor_projections_np = torch.repeat_interleave(neighbor_projections, len(negative_projections), dim=0)
                negative_projections_np = negative_projections.repeat(len(negative_projections), 1)
                l_text = angular_loss(anchors=positive_projections_np, positives=neighbor_projections_np, negatives=negative_projections_np)
                # print("after text angular loss")

                # Symmetric angular loss npairs (text to image)
                positive_projections_np = torch.repeat_interleave(positive_projections, len(image_projections), dim=0)
                image_projections_np = torch.repeat_interleave(image_projections, len(image_projections), dim=0)                
                permute_idxs = torch.from_numpy(np.asarray([j if i != j else (j+1) % len(image_projections) for i in range(len(image_projections)) for j in range(len(image_projections))]))
                image_projections_np2 = image_projections[permute_idxs,...]
                l_sym = angular_loss(anchors=positive_projections_np, positives=image_projections_np, negatives=image_projections_np2)
                # print("after t2i angular loss")

                # Bias loss scores
                # Bias loss on both text/img?
                # print("after bias loss")
                image_projections_np = torch.repeat_interleave(image_projections, len(negative_projections), dim=0)
                neighbor_imgs_projections_np = torch.repeat_interleave(bias_neighbor_imgs_projections, len(negative_projections), dim=0)
                permute_idxs = torch.from_numpy(np.asarray([j if i != j else (j+1) % len(image_projections) for i in range(len(image_projections)) for j in range(len(image_projections))]))
                image_projections_np2 = image_projections[permute_idxs,...]
                l_bias = angular_loss(anchors=image_projections_np, positives=neighbor_imgs_projections_np, negatives=image_projections_np2)
                
                # print(image_projections_np)
                # print(neighbor_imgs_projections_np)
                # print(permute_idxs)
                # print(image_projections_np2)
                
                # exit()


                # loss = l_i2t + float(sys.argv[1])*l_sym + float(sys.argv[2])*l_img + float(sys.argv[3])*l_text
                loss = l_i2t + float(T2I_WEIGHT)*l_sym + float(I_WEIGHT)*l_img + float(T_WEIGHT)*l_text + float(B_WEIGHT)*l_bias
                print(f"loss: {loss}")
                loss.backward()
                # grad_clip_value = 5
                # torch.nn.util.clip_grad_norm_(clipmodel.parameters(), grad_clip_value)
                optimizer.step()
                writer.add_scalar('data/train_loss', loss.item(), itr)
                writer.add_scalar('data/l_i2t', l_i2t.item(), itr)
                writer.add_scalar('data/l_sym', l_sym.item(), itr)
                writer.add_scalar('data/l_img', l_img.item(), itr)
                writer.add_scalar('data/l_text', l_text.item(), itr)

                pbar.update()

                # if (itr % 3)==0:
                #   break

        img_model.eval()
        rnn_model.eval()
        clipmodel.eval()
        losses = []
        random.seed(9485629)
        with tqdm(total=len(test_dataloader), ascii=True, leave=False, desc='eval') as pbar, torch.no_grad():
            for i, (images, paths, _, _, _, _) in enumerate(test_dataloader):
                optimizer.zero_grad()
                if (torch.cuda.is_available()):
                  images = images.float().cuda()
                # image_projections = img_model(images) # Batch size x 256
                image_projections = clipmodel.encode_image(images)

                # positive_sentences = [torch.from_numpy(MyDataset.result_db[path]).long() for path in paths]
                # pos_lengths = torch.LongTensor([torch.numel(item) for item in positive_sentences])
                # positive_sentences = torch.nn.utils.rnn.pad_sequence(sequences=positive_sentences, batch_first=True, padding_value=word_to_index['<!END!>']).cuda()
                positive_sentences = [MyDataset.result_db[local_from_impath(path)] for path in paths]

                # negative_sentences = [torch.from_numpy(MyDataset.result_db[path]).long() for path in random.sample(test_dataloader.dataset.dataset, len(positive_sentences))]
                # neg_lengths = torch.LongTensor([torch.numel(item) for item in negative_sentences])
                # negative_sentences = torch.nn.utils.rnn.pad_sequence(sequences=negative_sentences, batch_first=True, padding_value=word_to_index['<!END!>']).cuda()
                negative_sentences = [MyDataset.result_db[path] for path in random.sample(test_dataloader.dataset.dataset, len(positive_sentences))]

                # positive_projections = rnn_model(positive_sentences, pos_lengths)
                # negative_projections = rnn_model(negative_sentences, neg_lengths)
                positive_sentences = clip.tokenize(positive_sentences)
                negative_sentences = clip.tokenize(negative_sentences)

                if (torch.cuda.is_available()):
                  positive_sentences = positive_sentences.cuda()
                  negative_sentences = negative_sentences.cuda()

                positive_projections = clipmodel.encode_text(positive_sentences)
                negative_projections = clipmodel.encode_text(negative_sentences)

                loss = angular_loss(anchors=image_projections, positives=positive_projections, negatives=negative_projections)
                
                losses.append(loss.item())
                # print(f"loss: {loss}")

                pbar.update()
                # if i==3:
                #   break
        curr_loss = np.mean(losses)
        writer.add_scalar('data/val_loss', curr_loss, e)
        scheduler.step(curr_loss)
        try:
            torch.save({
              'epoch': e,
              'model_state_dict': clipmodel.state_dict(),
              'optimizer_state_dict': optimizer.state_dict(),
              'loss': curr_loss,
            }, f'/content/drive/MyDrive/Research/Political Image Debiasing/models/l_i2t_l_sym_{str(T2I_WEIGHT).replace(".","_")}_l_img_{str(I_WEIGHT).replace(".","_")}_l_text_{str(T_WEIGHT).replace(".","_")}_l_bias_{str(B_WEIGHT).replace(".","_")}/clip_model{e}.pt') #just change to your preferred folder/filename
            # torch.save(rnn_model.state_dict(), f'models/l_i2t_l_sym_{sys.argv[1].replace(".","_")}_l_img_{sys.argv[2].replace(".","_")}_l_text_{sys.argv[3].replace(".","_")}/rnn_model_{e}.pth')
            # torch.save(img_model.state_dict(), f'models/l_i2t_l_sym_{sys.argv[1].replace(".","_")}_l_img_{sys.argv[2].replace(".","_")}_l_text_{sys.argv[3].replace(".","_")}/img_model_{e}.pth')
        except Exception as exc:
            print('Failed saving epoch model')
            continue
        # save checkpoint of best model as well
        # print(f"curr loss: {curr_loss},  best_loss: {best_loss}")
        # print(f"losses: {losses}")
        if curr_loss < best_loss:
            best_loss = curr_loss
            f = open( f'/content/drive/MyDrive/Research/Political Image Debiasing/models/l_i2t_l_sym_{str(T2I_WEIGHT).replace(".","_")}_l_img_{str(I_WEIGHT).replace(".","_")}_l_text_{str(T_WEIGHT).replace(".","_")}_l_bias_{str(B_WEIGHT).replace(".","_")}/best_loss.txt', "w")
            f.write(str(best_loss))
            f.close()
            # # delete prior models
            # prior_models = glob.glob(f'models/l_i2t_l_sym_{str(T2I_WEIGHT).replace(".","_")}_l_img_{str(I_WEIGHT).replace(".","_")}_l_text_{str(T_WEIGHT).replace(".","_")}_l_bias_{str(B_WEIGHT).replace(".","_")}/*.pth')
            # for pm in prior_models:
            #     os.remove(pm)
            try:
                torch.save({
                  'epoch': e,
                  'model_state_dict': clipmodel.state_dict(),
                  'optimizer_state_dict': optimizer.state_dict(),
                  'loss': curr_loss,
                }, f'/content/drive/MyDrive/Research/Political Image Debiasing/models/l_i2t_l_sym_{str(T2I_WEIGHT).replace(".","_")}_l_img_{str(I_WEIGHT).replace(".","_")}_l_text_{str(T_WEIGHT).replace(".","_")}_l_bias_{str(B_WEIGHT).replace(".","_")}/clip_model_best.pt') #just change to your preferred folder/filename
                # torch.save(rnn_model.state_dict(), f'models/l_i2t_l_sym_{sys.argv[1].replace(".","_")}_l_img_{sys.argv[2].replace(".","_")}_l_text_{sys.argv[3].replace(".","_")}/rnn_model_{e}.pth')
                # torch.save(img_model.state_dict(), f'models/l_i2t_l_sym_{sys.argv[1].replace(".","_")}_l_img_{sys.argv[2].replace(".","_")}_l_text_{sys.argv[3].replace(".","_")}/img_model_{e}.pth')
            except Exception as exc:
                print('Failed saving best model')
                continue
if __name__ == '__main__':
    main()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## Retrieval

In [17]:
retrieval_space_file = "/content/drive/MyDrive/Research/Political Image Debiasing/retrieval_space"
train_dataloader = torch.utils.data.DataLoader(dataset=MyDataset('train', "/content/drive/MyDrive/Research/Political Image Debiasing/dataset/dataset_metadata_clean_5000.pickle"), batch_size=1, shuffle=True, num_workers=NUM_WORKERS)
val_dataset = MyDataset('val', "/content/drive/MyDrive/Research/Political Image Debiasing/dataset/dataset_metadata_clean_5000.pickle")
test_dataloader = torch.utils.data.DataLoader(dataset=val_dataset, batch_size=1, shuffle=False, num_workers=NUM_WORKERS)

In [18]:
# Must run once
import matplotlib.pyplot as plt
def make_retrieval_space():
    space = []
    clipmodel, clippreprocess = get_clip()
    train_test_dict = pickle.load(open(split_paths, 'rb'))
    train_set = train_test_dict['train']
    # test_set = train_test_dict['test']
    # val_set = set(random.sample(train_set, len(train_test_dict['test'])))
    # train_set = [pth for pth in train_set if pth not in val_set]
    # Get the features for each image in the training set
    clipmodel.eval()
    for i, (images, paths, _, _, _, _) in tqdm(enumerate(train_dataloader)):
      if (torch.cuda.is_available()):
        images = images.float().cuda()
      # Encode img
      image_projection = clipmodel.encode_image(images)
      encoding = image_projection.numpy(force=True)[0]
      # Get bias for each image
      key_path = local_from_impath(paths[0])
      bias = bias_from_idx(_neighbors_pths2idx[key_path])
      space.append((key_path, bias, encoding))
    # Write to file
    with open(retrieval_space_file, "wb") as f:
      pickle.dump(space, f)

make_retrieval_space()

100%|████████████████████████████████████████| 338M/338M [00:02<00:00, 119MiB/s]


Loaded model state dict: /content/drive/MyDrive/Research/Political Image Debiasing/models/l_i2t_l_sym_1_l_img_0_3_l_text_0_2_l_bias_0_3/clip_model3.pt


87it [00:49,  1.76it/s]


KeyboardInterrupt: ignored

In [ ]:
f = open(retrieval_space_file, "rb")
space = pickle.load(f)
f.close()
print(space)

# clipmodel, clippreprocess = get_clip()
# def retrieve_debiased_image_from_text(text):
#     # Encode text
#     texttokens = clipmodel.tokenize(text)
#     text_encodings = clipmodel.tokenize(text)

In [ ]:
import gc
torch.cuda.empty_cache()
gc.collect()

In [ ]:
!pip list

In [ ]:
!conda list

In [ ]:
print(torch.__version__)